Libraries

In [23]:
import pandas as pd
import os
from datetime import datetime


Data

In [7]:
###### flight api data
# set path
data_directory_name = f'../../Data/FlightAPIData/'
subfolder_name = f'2023-04-26_03-36-24/'
path_name = f'{data_directory_name}{subfolder_name}'

# init data frames
data_frames = []

# get all data ames
for filename in os.listdir(path_name):
    if filename.endswith('.csv'):
        file_path = os.path.join(path_name, filename)
        df = pd.read_csv(file_path)
        data_frames.append(df)

# concat
flight_data = pd.concat(data_frames, ignore_index=True)


###### capital cities data
capital_cities = pd.read_csv('../../Data/capital_cities_and_airports.csv')
iata_codes_list = capital_cities['airport_iata'].tolist()


In [9]:
# check
display(flight_data.shape)
display(flight_data.head())

(25036, 18)

,id,price_amount,price_unit,segment_id,transfer_type,agent_name,duration_minutes,marketing_flight_number,departure_datetime,arrival_datetime,marketing_carrier_name,operating_carrier_name,origin_place_name,origin_place_iata,destination_place_name,destination_place_iata,departure_query,arrival_query
0,"15458-2305221540--32166,-32348,-32301-2-11431-...",644800.0,PRICE_UNIT_MILLI,15458-16292-2305221540-2305221845--32166,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Trip.com,125,594,2023-05-22 15:40:00,2023-05-22 18:45:00,Jetstar,Jetstar Asia,Phnom Penh,PNH,Singapore Changi,SIN,PNH,EVN
1,"15458-2305221540--32166,-32348,-32301-2-11431-...",644800.0,PRICE_UNIT_MILLI,16292-11182-2305222110-2305230050--32348,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Trip.com,460,355,2023-05-22 21:10:00,2023-05-23 00:50:00,Emirates,Emirates,Singapore Changi,SIN,Dubai,DXB,PNH,EVN
2,"15458-2305221540--32166,-32348,-32301-2-11431-...",644800.0,PRICE_UNIT_MILLI,11182-11431-2305230800-2305231115--32301,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Trip.com,195,717,2023-05-23 08:00:00,2023-05-23 11:15:00,flydubai,flydubai,Dubai,DXB,Yerevan,EVN,PNH,EVN
3,"15458-2305221540--32166,-32348-2-11431-2305231115",807600.0,PRICE_UNIT_MILLI,15458-16292-2305221540-2305221845--32166,TRANSFER_TYPE_MANAGED,Expedia,125,594,2023-05-22 15:40:00,2023-05-22 18:45:00,Jetstar,Jetstar Asia,Phnom Penh,PNH,Singapore Changi,SIN,PNH,EVN
4,"15458-2305221540--32166,-32348-2-11431-2305231115",807600.0,PRICE_UNIT_MILLI,16292-11182-2305222110-2305230050--32348,TRANSFER_TYPE_MANAGED,Expedia,460,355,2023-05-22 21:10:00,2023-05-23 00:50:00,Emirates,Emirates,Singapore Changi,SIN,Dubai,DXB,PNH,EVN


In [11]:
# get all values queried to get those that are missing

flight_data_complete = flight_data[['departure_query', 'arrival_query']]
flight_data_complete = flight_data_complete.drop_duplicates()
flight_data_complete.shape

(1222, 2)

In [17]:
# get all possible values
airport_pairs = [(iata_codes_list[i], iata_codes_list[j]) for i in range(len(iata_codes_list)) for j in range(len(iata_codes_list)) if i != j]
airport_pairs = pd.DataFrame(airport_pairs, columns=['departure_query', 'arrival_query']) # to dataframe

airport_pairs.shape

(1806, 2)

In [25]:
# the missing rows
# merge
merged_df = pd.merge(airport_pairs, flight_data_complete, on=['departure_query', 'arrival_query'], how='left', indicator=True)

# filter out non null
not_in_database = merged_df[merged_df['_merge'] == 'left_only']
not_in_database = not_in_database.reset_index()
not_in_database = not_in_database.drop(['_merge', 'index'], axis=1)

# display the result
display(not_in_database)

# save
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
not_in_database.to_csv(f'../../Data/FlightAPIData/PendingValues/{timestamp}_pending_values.csv', index=False)

,departure_query,arrival_query
0,KBL,EVN
1,KBL,GYD
2,KBL,BAH
3,KBL,DAC
4,KBL,BWN
...,...,...
579,HAN,MCT
580,HAN,ISB
581,HAN,RUH
582,HAN,CMB
